In [ ]:
# change into the root directory of the project
import os
if os.getcwd().split("/")[-1] == "dev":
    os.chdir('..')
    
# get the current directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))


# Set path, where to save files of tables
PATH = cwd
HDF_PATH = PATH + '/data/hdf/'
IMG_PATH = PATH + '/images/05/'
print(HDF_PATH)


import numpy as np
import scipy
from scipy import stats


from neurolib.models.wc-adap import WCModel
from neurolib.models.wc_input import WCModel_input

from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.exploration import BoxSearch
import neurolib.utils.functions as func
import neurolib.utils.devutils as du

#import neurolib.utils.brainplot as bp
import neurolib.optimize.exploration.explorationUtils as eu

from neurolib.utils.loadData import Dataset

from Derivations import Derivations

#for the brainplot functions
from Brainplot import plot_brain
from Brainplot import plot_involvement_distribution
import xarray as xr
from neurolib.utils.signal import Signal
import matplotlib.pyplot as plt

# Initialize the model

In [ ]:
ds = Dataset("gw", fcd=True)
wc = WCModel(Cmat = ds.Cmat, Dmat = ds.Dmat)
wc_input=WCModel_input(Cmat = ds.Cmat, Dmat = ds.Dmat)

In [ ]:
fix = {'duration': 240.0*1000, 'sigma_ou': 0.287031, 'K_gl': 2.573845, 
      # 'adap_init': 0.0 * np.random.uniform(0.99, 1, (1, 1)), #inkl a_adap Null setzen für ausschalten der Adap
       'tau_exc': 2.5, 'tau_inh': 3.75, 
       'c_excexc': 16, 'c_excinh': 12, 'c_inhexc': 12, 'c_inhinh': 3, 
       'a_exc': 1.0, 'a_inh': 1.0, 
       'mu_exc': 5.0, 'mu_inh': 5.0,
       'a_adap': 98.381822, 'tau_adap': 3050.402224,
       'exc_ext': 4.960871, 'inh_ext': 2.668888}
wc.params.update(fix)

In [ ]:
dev = Derivations(model=wc, model_input=wc_input, params=fix)

In [ ]:
ParamSpace = {'exc_ext': [3.96], 'inh_ext': [2.77]}
print('Parameters Space: ', ParamSpace)

#HDF_PATH is set in second cell
filestr = '01_Waves_fitting results.hdf'
search = dev.runToSave(HDF_PATH, filestr, ParamSpace, False)

In [ ]:
search.loadResults()
df = search.dfResults

# Prepare the data

In [ ]:
#Packages for plotting
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.offline as pyo

import chart_studio.plotly as py #chart_studio has to be installed: pip install chart_studio
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px

from Templates import template
from Templates import symbols
from Templates import global_waves_color_list as colors


width=template.layout.width*(1/3)
height=template.layout.height*(2/3)


In [ ]:
#1. The developement of global waves per noise for increasing b

fig = go.Figure()


for i, (key, val) in enumerate(df.groupby(['sigma_ou'])):
    proportion = val['peaks50'].rolling(3, min_periods=1).mean()
   # print(proportion)
    fig.add_trace(go.Scatter(x=val['a_adap'], y=proportion, 
                             mode='lines',
                             marker=dict(color=colors[i]),
                             name=key,
                            showlegend=False))
    
fig.update_layout(template=template,
                  annotations=[
                      dict(x=-0.3,y=0.5, text='Global waves [1/min]', 
                           font_size=26, font_color='black',textangle=-90),
                      dict(x=-0.29,y=1.15, text='(a)', font_color='black')
                  ],
                 width=width, height=height,
                 xaxis=dict(title_text='',#'Adaptation strength b',
                            ticks='outside', tickmode='array',
                           tickvals=[0, 50, 98.38, 150], ticktext=['0','50',r'$b^c$','150']),
                 yaxis=dict(title_text='', ticks='outside',
                           tickvals=[0, 100, 200]),
                 margin=dict(l=80))

fig.show()

fig.write_image(IMG_PATH + 'global.png', scale=1.0)

In [ ]:
#2. The developement of local waves per noise for increasing b

fig = go.Figure()


for i, (key, val) in enumerate(df.groupby(['sigma_ou'])):
    proportion = (val['peaks25']-val['peaks50']).rolling(3, min_periods=1).mean()
   # print(proportion)
    fig.add_trace(go.Scatter(x=val['a_adap'], y=proportion, 
                             mode='lines',
                             marker=dict(color=colors[i]),
                             name=key,
                            showlegend=False))
    
fig.update_layout(template=template,
                  annotations=[
                      dict(x=-0.2,y=1.15, text='(b)', font_color='black')
                  ],
                 width=width, height=height,
                 xaxis=dict(title_text='',#'Adaptation strength b',
                            ticks='outside', tickmode='array',
                           tickvals=[0, 50, 98.38, 150], ticktext=['0','50',r'$b^c$','150'], range=[50,150]),
                 yaxis=dict(title_text='Local waves [1/min]'))

fig.show()
fig.write_image(IMG_PATH + 'local.png', scale=1.5)

In [ ]:
# 3. up-down durations per noise, over increasing b

#1. The developement of global waves per noise for increasing b

fig = go.Figure()

from Templates import brain_result_colors as colors_dict


for i, (key, val) in enumerate(df.groupby(['sigma_ou'])):
    proportion_down = val['down_duration'].rolling(3, min_periods=1).mean()
    proportion_up = val['up_duration'].rolling(3, min_periods=1).mean()
   # print(proportion)
    fig.add_trace(go.Scatter(x=val['a_adap'], y=proportion_down, 
                             mode='lines',
                             marker=dict(color=colors[i]),
                             name=key, showlegend=False))
    fig.add_trace(go.Scatter(x=val['a_adap'], y=proportion_up, 
                             mode='lines',
                             marker=dict(color=colors[i]),
                             line=dict(dash='dash'),
                             name=key, showlegend=False))
    

fig.add_trace(go.Scatter(x=[0], y=[0], 
                         mode='lines',
                         marker=dict(color='black', symbol=symbols[0]), 
                         visible='legendonly', showlegend=True,
                         name='down'))
fig.add_trace(go.Scatter(x=[0], y=[0], 
                         mode='lines',
                         line=dict(dash='dash'),
                         marker=dict(color='black', symbol=symbols[0]), 
                         visible='legendonly', showlegend=True,
                         name='up'))
    
fig.update_layout(template=template,
                  annotations=[
                      dict(x=-0.2,y=1.16, text='(c)', font_color='black')
                  ],
                 width=width, height=height,
                 xaxis=dict(title_text='Adaptation strength b', range=[60,150],
                            ticks='outside', tickmode='array',
                           tickvals=[0, 60, 98.38, 150], ticktext=['0','60',r'$b^c$','150']),
                 yaxis=dict(title_text='State Durations [s]', range=[0,10]),
                 legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.show()
fig.write_image(IMG_PATH + 'state_dur.png', scale=1.5)

In [ ]:
# 4. The evolution of dominant frequency per sigma_ou w.r.t. b

fig = go.Figure()


for i, (key, val) in enumerate(df.groupby(['sigma_ou'])):
    proportion = val['domfrequ'].rolling(3, min_periods=1).mean()
    fig.add_trace(go.Scatter(x=val['a_adap'], y=proportion,
                             mode='lines',
                             marker=dict(color=colors[i]),
                             name=key))
    
fig.update_layout(template=template,
                  annotations=[
                      dict(x=-0.2,y=1.15, text='(d)', font_color='black')
                  ],
                 width=width+60, height=height,
                 xaxis=dict(title_text='Adaptation strength b',
                            ticks='outside', tickmode='array',
                            tickvals=[0, 50, 98.38, 150], ticktext=['0','50',r'$b^c$','150']),
                 yaxis=dict(title_text='Frequency [Hz]', tickvals=[0,10,20,30]),
                 legend=dict(title='noise'))

fig.show()
fig.write_image(IMG_PATH + 'frequ.png', scale=1.5)

In [ ]:
df

In [ ]:
wc.params.exc_ext = 4.5
wc.params.inh_ext = 3.77

wc.params.a_adap = 0.0
wc.params.sigma_ou = 0.0

wc.run()

x = wc.exc[:,-50000:]

fig = go.Figure()

for i in [0,5,10,20,31,40,50]:
    fig.add_trace(go.Scatter(x=np.linspace(0,5,50000), y=x[i], name=i))
    
fig.update_layout(template=template,
                 width=600, height=300,
                 yaxis=dict(range=[0,1.01]))

fig.show()

In [ ]:
states = [dev.getUpDownWindows(xss) for xss in x]
states

In [ ]:
states = np.array(states)
stateLengths = [dev.getStateLengths(xs) for xs in states]
stateLengths

In [ ]:
durations = dev.getDurationsNdim(stateLengths)
durations